# IST 718: Big Data Analytics

- Professor: Willard Williamson <wewillia@syr.edu>
- Faculty Assistant: Palaniappan Muthukkaruppan
## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers from your classmates.  Short code snippets are allowed from the internet.  Any code is allowed from the class text books or class provided code.__
- Please do not change the file names. The FAs and the professor use these names to grade your homework.
- Remove or comment out code that contains `raise NotImplementedError`. This is mainly to make the `assert` statement fail if nothing is submitted.
- The tests shown in some cells (i.e., `assert` and `np.testing.` statements) are used to grade your answers. **However, the professor and FAs will use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`).

In [2]:
# Load the packages needed for this part
# create spark and sparkcontext objects
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline, classification, pipeline, evaluation
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import functions as fn, Row
from pyspark.sql.functions import when, regexp_extract, col
from pyspark import sql

import matplotlib.pyplot as plt
import pandas as pd

In this section, you are going to develop a SMS spam detector based on logistic regression. This is the same idea behind sentiment analysis, but instead of predicting positive sentiment vs negative sentiment, you are going to predict whether a SMS text is spam or not.

The dataset will be in `sms_spam_df`

In [4]:
sms_spam_df = spark.read.csv('/FileStore/tables/sms_spam.csv', header=True, inferSchema=True)

# Question 1.1 (10 pts)

Encode the `type` column to be 1 for `spam` and 0 for `ham` and store the result in `sms_spam2_df`

In [6]:
# create sms_spam2_df below
sms_spam2_df = sms_spam_df.withColumn("type", fn.when(fn.col('type') == ('spam'),1).otherwise(0))
sms_spam2_df.show()
#raise NotImplementedError()

+----+--------------------+
type| text|
+----+--------------------+
 0|Go until jurong p...|
 0|Ok lar... Joking ...|
 1|Free entry in 2 a...|
 0|U dun say so earl...|
 0|Nah I don't think...|
 1|FreeMsg Hey there...|
 0|Even my brother i...|
 0|As per your reque...|
 1|WINNER!! As a val...|
 1|Had your mobile 1...|
 0|I'm gonna be home...|
 1|SIX chances to wi...|
 1|URGENT! You have ...|
 0|I've been searchi...|
 0|I HAVE A DATE ON ...|
 1|XXXMobileMovieClu...|
 0|Oh k...i'm watchi...|
 0|Eh u remember how...|
 0|Fine if that's th...|
 1|England v Macedon...|
+----+--------------------+
only showing top 20 rows

In [7]:
# (10 pts)
np.testing.assert_array_equal(
    sms_spam2_df.groupBy('type').count().orderBy('type').rdd.map(lambda x: x['count']).collect(),
    [4827, 747]
)

# Question 1.2: tfidf feature engineering (15 pts)
Create a pipeline that combines a `Tokenizer`, `CounterVectorizer`, and a `IDF` estimator to compute the tfidf vectors of each SMS. Fit this pipeline and assign the pipeline transformer to a variable `tfidf_pipeline`. The `Tokenizer` step should create a column `words`, the `CounterVectorizer` step should create a column `tf`, and the `IDF` step should create a column `tfidf`.

In [9]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF

In [10]:
# create a Pipeline transformer and name it tfidf_pipeline
tokenizer = Tokenizer().setInputCol('text').setOutputCol('words')
CounterVectorizer = CountVectorizer().setInputCol('words').setOutputCol('tf')
IDF = IDF().setInputCol('tf').setOutputCol('tfidf')
tfidf_pipeline = Pipeline(stages = [tokenizer, CounterVectorizer, IDF]).fit(sms_spam2_df)
#raise NotImplementedError()

In [11]:
tfidf_pipeline.transform(sms_spam2_df).show(5)

+----+--------------------+--------------------+--------------------+--------------------+
type| text| words| tf| tfidf|
+----+--------------------+--------------------+--------------------+--------------------+
 0|Go until jurong p...|[go, until, juron...|(13525,[8,42,51,6...|(13525,[8,42,51,6...|
 0|Ok lar... Joking ...|[ok, lar..., joki...|(13525,[5,74,404,...|(13525,[5,74,404,...|
 1|Free entry in 2 a...|[free, entry, in,...|(13525,[0,3,8,20,...|(13525,[0,3,8,20,...|
 0|U dun say so earl...|[u, dun, say, so,...|(13525,[5,22,60,1...|(13525,[5,22,60,1...|
 0|Nah I don't think...|[nah, i, don't, t...|(13525,[0,1,66,86...|(13525,[0,1,66,86...|
+----+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [12]:
# (15 pts)
np.testing.assert_array_equal([type(s) for s in tfidf_pipeline.stages],
                              [feature.Tokenizer, feature.CountVectorizerModel, feature.IDFModel])

Investigate the fitted pieline above and create a variable `lowest_idf` that contains the set of words with the 5 lowest IDF. **Hint: you must extract the vocabulary from the fitted `CountVectorizer` and the IDF values from the fitted `IDF`, both in the stages of `tfidf_pipeline`. You can put both lists into Pandas dataframe columns and sort by idf, picking 5 after sorting**

In [14]:
tfidf_pd_df = tfidf_pipeline.transform(sms_spam2_df).toPandas()
tfidf_pd_df.head()

,type,text,words,tf,tfidf
0,0,"Go until jurong point, crazy.. Available only ...","[go, until, jurong, point,, crazy.., available...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.975..."
1,0,Ok lar... Joking wif u oni...,"[ok, lar..., joking, wif, u, oni...]","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 2.02617709712, 0.0, ..."
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","(3.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(3.60564694586, 0.0, 0.0, 1.56987231223, 0.0, ..."
3,0,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor..., u, c, already...","(0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 4.05235419423, 0.0, ..."
4,0,"Nah I don't think he goes to usf, he lives aro...","[nah, i, don't, think, he, goes, to, usf,, he,...","(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.20188231529, 1.25016544811, 0.0, 0.0, 0.0, ..."


In [15]:
# create lower_idf with list of words with 5 lowest IDF values.
vocab = tfidf_pipeline.stages[1].vocabulary
idf = tfidf_pipeline.stages[-1].idf.toArray()
lowest_idf = pd.DataFrame({'Vocab': vocab, 'IDF': idf})
lowest_idf = lowest_idf.sort_values('IDF')
lowest_idf = set(lowest_idf.Vocab.head())
#raise NotImplementedError()

In [16]:
# (5 pts)
# it is a set
np.testing.assert_equal(type(lowest_idf), set)
# it has 5 elements
np.testing.assert_equal(len(lowest_idf), 5)
# each element is a string
np.testing.assert_equal({type(w) for w in lowest_idf}, {str})

# Question 1.3: uppercase feature (15 pts)

Typical spam messages contain words that are upper case. Create a dataframe `sms_spam3_df` where you add a new column `has_uppercase` which contains an integer `1` if the first sequence of uppercase letters is longer or equal to 3 and an integer `0` otherwise.

In [18]:
pd.options.display.max_colwidth = 500
pd.set_option('display.max_rows', None)

In [19]:
# create sms_spam3_df below
sms_spam3_df = sms_spam2_df.select(fn.col('type'), fn.col('text'), fn.when((fn.length(fn.regexp_extract(fn.col('text'),'[A-Z]{3,}',0))) >= 3,1).otherwise(0).alias('has_uppercase'))
#raise NotImplementedError()

In [20]:
sms_spam3_df.show()

+----+--------------------+-------------+
type| text|has_uppercase|
+----+--------------------+-------------+
 0|Go until jurong p...| 0|
 0|Ok lar... Joking ...| 0|
 1|Free entry in 2 a...| 0|
 0|U dun say so earl...| 0|
 0|Nah I don't think...| 0|
 1|FreeMsg Hey there...| 0|
 0|Even my brother i...| 0|
 0|As per your reque...| 0|
 1|WINNER!! As a val...| 1|
 1|Had your mobile 1...| 1|
 0|I'm gonna be home...| 0|
 1|SIX chances to wi...| 1|
 1|URGENT! You have ...| 1|
 0|I've been searchi...| 0|
 0|I HAVE A DATE ON ...| 1|
 1|XXXMobileMovieClu...| 1|
 0|Oh k...i'm watchi...| 0|
 0|Eh u remember how...| 0|
 0|Fine if that's th...| 0|
 1|England v Macedon...| 1|
+----+--------------------+-------------+
only showing top 20 rows

The first three messages with `has_uppercase == 1` are as follows:

```python
sms_spam3_df.where('has_uppercase == 1').take(3)
```

```console
[Row(type=1, text='WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.', has_uppercase=1),
 Row(type=1, text='Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030', has_uppercase=1),
 Row(type=1, text='SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info', has_uppercase=1)]
```

In [22]:
# try it here
sms_spam3_df.where('has_uppercase == 1').take(3)

Out[ 38 ]: [Row(has_uppercase=1, text='WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.', type=1),
 Row(has_uppercase=1, text='Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030', type=1),
 Row(has_uppercase=1, text='SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info', type=1)]

In [23]:
# (15 pts)
np.testing.assert_equal(set(sms_spam3_df.columns), {'has_uppercase', 'text', 'type'})
np.testing.assert_equal(type(sms_spam3_df.schema['has_uppercase'].dataType), sql.types.IntegerType)
np.testing.assert_equal(sms_spam3_df.rdd.map(lambda x : x['has_uppercase']).sum(), 891)

# Question 1.4: Compare models (15 pts)

Using the following splits:

In [25]:
training_df, validation_df, testing_df = sms_spam2_df.randomSplit([0.6, 0.3, 0.1], seed=0)

In [26]:
[training_df.count(), validation_df.count(), testing_df.count()]

Out[ 41 ]: [3349, 1674, 551]

Create pipelines where the first stage is the `tfidf_pipeline` created above and the second stage is a `LogisticRegression` model with different regularization parameters ($\lambda$) and elastic net mixture ($\alpha$). Fit those pipelines to the appropriate data split.

1. Logistic regression with $\lambda=0$ and $\alpha=0$ (assign the fitted pipeline to `lr_pipeline1`)
2. Logistic regression with $\lambda=0.02$ and $\alpha=0.2$ (assign the fitted pipeline to `lr_pipeline2`)
3. Logistic regression with $\lambda=0.1$ and $\alpha=0.4$ (assign the fitted pipeline to `lr_pipeline3`)

In [28]:
# create lr_pipeline1, lr_pipeline2, and lr_pipeline3
lr1 = LogisticRegression().setLabelCol('type').setFeaturesCol('tfidf').setRegParam(0.0).setMaxIter(100).setElasticNetParam(0.0)
lr2 = LogisticRegression().setLabelCol('type').setFeaturesCol('tfidf').setRegParam(0.02).setMaxIter(100).setElasticNetParam(0.2)
lr3 = LogisticRegression().setLabelCol('type').setFeaturesCol('tfidf').setRegParam(0.1).setMaxIter(100).setElasticNetParam(0.4)
lr_pipeline1 = Pipeline(stages=[tfidf_pipeline, lr1]).fit(training_df)
lr_pipeline2 = Pipeline(stages=[tfidf_pipeline, lr2]).fit(training_df)
lr_pipeline3 = Pipeline(stages=[tfidf_pipeline, lr3]).fit(training_df)
#raise NotImplementedError()

In [29]:
# (15 pts)
np.testing.assert_equal(type(lr_pipeline1), pipeline.PipelineModel)
np.testing.assert_equal(type(lr_pipeline2), pipeline.PipelineModel)
np.testing.assert_equal(type(lr_pipeline3), pipeline.PipelineModel)
np.testing.assert_array_equal([type(s) for s in lr_pipeline1.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])
np.testing.assert_array_equal([type(s) for s in lr_pipeline2.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])
np.testing.assert_array_equal([type(s) for s in lr_pipeline3.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])

Use the evaluator object defined below to compute the ROC AUC of your predictors. For example, to compute the AUC of pipeline 1 for a dataframe `df`, you would run

```python
evaluator.evaluate(lr_pipeline1.transform(df))
```

Assign the AUC of the three models to the variables `AUC1`, `AUC2`, and `AUC3`, and and assign the pipeline with the best model to a variable `best_model`

In [31]:
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='type')

For example, the AUC on training of the first model is perfect:

```
evaluator.evaluate(lr_pipeline1.transform(training_df))
```

```console
1.0
```

In [33]:
# AUC code here
AUC1 = evaluator.evaluate(lr_pipeline1.transform(validation_df))
AUC2 = evaluator.evaluate(lr_pipeline2.transform(validation_df))
AUC3 = evaluator.evaluate(lr_pipeline3.transform(validation_df))
#raise NotImplementedError()

In [34]:
# (15 pts)
np.testing.assert_array_equal([type(AUC1), type(AUC2), type(AUC3)],
                             [float, float, float])
# AUC less than 1
np.testing.assert_array_less([AUC1, AUC2, AUC3], [1, 1, 1])
# AUC more than 0.5
np.testing.assert_array_less([.5, .5, .5],
                            [AUC1, AUC2, AUC3])

In [35]:
print(AUC1, AUC2, AUC3)

0.9557218489415972 0.9871555611031791 0.9686667539402503

In [36]:
# Create best_model variable here
best_model = lr_pipeline2

# Question 1.5: Choose best model (15 pts)

Using the right split and the best model selected before, compute the generalization performance and assign it to a variable `AUC_best`

In [38]:
# Compute the AUC of the best model and assign to the variable AUC_best
AUC_best = evaluator.evaluate(best_model.transform(testing_df))
#raise NotImplementedError()

In [39]:
# (15 pts)
np.testing.assert_approx_equal(AUC_best, 
                               0.976126746201693, significant=2)

# Question 1.6: Inference (15 pts)

Use the pipeline 2 fitted above (`lr_pipeline2`) to create Pandas dataframes that contain the most negative words and the most positive words. In particular, create a dataframe `positive_words` with the columns `word` and `weight` with the top 20 positive words, sorted by descending coefficient. Similarly create a `negative_words` Pandas dataframe with the top 20 negative words where the coefficient are sorted in ascending order. **Hint: Use the `sentiment_analysis.ipynb` notebook in the repo for inspiration**

In [41]:
# create positive_words and negative_words pandas dataframe below    
vocabulary = tfidf_pipeline.stages[1].vocabulary
weights = lr_pipeline2.stages[-1].coefficients.toArray()
coeffs_df = pd.DataFrame({'word': vocabulary, 'weight': weights})
positive_words = coeffs_df.sort_values('weight', ascending=False).head(20)
negative_words = coeffs_df.sort_values('weight', ascending=True).head(20)
#raise NotImplementedError()

In [42]:
positive_words.head()

,weight,word
3555,0.590870,widelive.com/index.
12237,0.533567,08714712388
15,0.517100,call
81,0.513278,txt
9064,0.468274,gbp/sms


In [43]:
negative_words.head()

,weight,word
1,-0.162493,i
2444,-0.060939,fighting
3221,-0.059061,dificult
3371,-0.059061,fightng
3332,-0.059061,lose.


The `positive_words` and `negative_words` dataframe should look like this:

```python
positive_words.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>3555</th>
      <td>widelive.com/index.</td>
      <td>0.590870</td>
    </tr>
    <tr>
      <th>12237</th>
      <td>08714712388</td>
      <td>0.533567</td>
    </tr>
    <tr>
      <th>15</th>
      <td>call</td>
      <td>0.517100</td>
    </tr>
    <tr>
      <th>81</th>
      <td>txt</td>
      <td>0.513278</td>
    </tr>
    <tr>
      <th>9064</th>
      <td>gbp/sms</td>
      <td>0.468274</td>
    </tr>
  </tbody>
</table>

and 

```python
negative_words.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>i</td>
      <td>-0.162493</td>
    </tr>
    <tr>
      <th>2444</th>
      <td>fighting</td>
      <td>-0.060939</td>
    </tr>
    <tr>
      <th>3221</th>
      <td>dificult</td>
      <td>-0.059061</td>
    </tr>
    <tr>
      <th>3371</th>
      <td>fightng</td>
      <td>-0.059061</td>
    </tr>
    <tr>
      <th>3332</th>
      <td>lose.</td>
      <td>-0.059061</td>
    </tr>
  </tbody>
</table>

In [45]:
# (15 pts)
np.testing.assert_equal(set(positive_words.columns), {'weight', 'word'})
np.testing.assert_equal(set(negative_words.columns), {'weight', 'word'})
np.testing.assert_approx_equal(positive_words.weight.sum(), 8.3701485692317927, significant=2)
np.testing.assert_approx_equal(negative_words.weight.sum(), -0.6661952507442954, significant=2)
np.testing.assert_array_less(positive_words.weight.iloc[-1], positive_words.weight.iloc[0])
np.testing.assert_array_less(negative_words.weight.iloc[0], negative_words.weight.iloc[-1])

# 1.7: Inference (15 pts)
Use the dataframe `sms_spam3_df` to create a model where the first feature is `has_uppercase` and the next set of features are the tfidf of the text. Perform feature engineering in all features using a max absolute scaler ([`MaxAbsScaler`](https://spark.apache.org/docs/2.0.2/ml-features.html#maxabsscaler)). Do a logistic regression on the resulting scaled features with regularization parameter $\lambda = 0.2$ and elastic net mixture $\alpha=0.1$ for the entire data (all of `sms_spam3_df`). Since you have scaled all features to be within the same range, you can compare them. 

**(5 pts)** with code and comments, answer below

1. is `has_uppercase` a feature that is positively or negative related to an SMS being spam?
2. what is the ratio of the coefficient of `has_uppercase` to the biggest positive tfidf coefficient?

In [47]:
# your code and comments below
# YOUR CODE HERE
va = feature.VectorAssembler(inputCols = ['has_uppercase', 'tfidf'], outputCol = 'features')
scale = feature.MaxAbsScaler(inputCol = 'features', outputCol = 'scaled_features')
scaled_lr = LogisticRegression().setLabelCol('type').setFeaturesCol('scaled_features').setRegParam(0.2).setMaxIter(100).setElasticNetParam(0.1)
scaled_pipeline = Pipeline(stages = [tokenizer, CounterVectorizer, IDF, va, scale, scaled_lr]).fit(sms_spam3_df)
#raise NotImplementedError()

In [48]:
scaled_pipeline.transform(sms_spam3_df).show()

+----+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
type| text|has_uppercase| words| tf| tfidf| features| scaled_features| rawPrediction| probability|prediction|
+----+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 0|Go until jurong p...| 0|[go, until, juron...|(13525,[8,42,51,6...|(13525,[8,42,51,6...|(13526,[9,43,52,6...|(13526,[9,43,52,6...|[2.79089435542085...|[0.94218178424803...| 0.0|
 0|Ok lar... Joking ...| 0|[ok, lar..., joki...|(13525,[5,74,404,...|(13525,[5,74,404,...|(13526,[6,75,405,...|(13526,[6,75,405,...|[2.79089435542085...|[0.94218178424803...| 0.0|
 1|Free entry in 2 a...| 0|[free, entry, in,...|(13525,[0,3,8,20,...|(13525,[0,3,8,20,...|(13526,[1,4,9,21,...|(13526,[1,4,9,21,...|[-0.4357532631849...|[0.39275334554252...| 1.0|
 0|U dun say so earl...| 0|[u, dun, say, so,...|(13525,[5,22,60,1...|(13525,[5,22,60,1...|(13526,[6,23,61,1...|(13526,[6,23,61,1...|[2.79089435542085...|[0.94218178424803...| 0.0|
 0|Nah I don't think...| 0|[nah, i, don't, t...|(13525,[0,1,66,86...|(13525,[0,1,66,86...|(13526,[1,2,67,87...|(13526,[1,2,67,87...|[2.75817342641400...|[0.94037329790613...| 0.0|
 1|FreeMsg Hey there...| 0|[freemsg, hey, th...|(13525,[0,2,6,10,...|(13525,[0,2,6,10,...|(13526,[1,3,7,11,...|(13526,[1,3,7,11,...|[2.12445996029986...|[0.89325792517319...| 0.0|
 0|Even my brother i...| 0|[even, my, brothe...|(13525,[0,7,9,13,...|(13525,[0,7,9,13,...|(13526,[1,8,10,14...|(13526,[1,8,10,14...|[2.69317790057607...|[0.93662288524559...| 0.0|
 0|As per your reque...| 0|[as, per, your, r...|(13525,[0,10,11,4...|(13525,[0,10,11,4...|(13526,[1,11,12,4...|(13526,[1,11,12,4...|[1.68978196261293...|[0.84419548368669...| 0.0|
 1|WINNER!! As a val...| 1|[winner!!, as, a,...|(13525,[0,2,3,14,...|(13525,[0,2,3,14,...|(13526,[0,1,3,4,1...|(13526,[0,1,3,4,1...|[-1.7959385100822...|[0.14234618692169...| 1.0|
 1|Had your mobile 1...| 1|[had, your, mobil...|(13525,[0,4,5,10,...|(13525,[0,4,5,10,...|(13526,[0,1,5,6,1...|(13526,[0,1,5,6,1...|[-1.1233525119169...|[0.24538995665625...| 1.0|
 0|I'm gonna be home...| 0|[i'm, gonna, be, ...|(13525,[0,1,6,29,...|(13525,[0,1,6,29,...|(13526,[1,2,7,30,...|(13526,[1,2,7,30,...|[2.75817342641400...|[0.94037329790613...| 0.0|
 1|SIX chances to wi...| 1|[six, chances, to...|(13525,[0,6,40,48...|(13525,[0,6,40,48...|(13526,[0,1,7,41,...|(13526,[0,1,7,41,...|[-0.3862113927038...|[0.40462966249821...| 1.0|
 1|URGENT! You have ...| 1|[urgent!, you, ha...|(13525,[0,2,3,4,8...|(13525,[0,2,3,4,8...|(13526,[0,1,3,4,5...|(13526,[0,1,3,4,5...|[-1.5702246897558...|[0.17218436279181...| 1.0|
 0|I've been searchi...| 0|[i've, been, sear...|(13525,[0,1,2,3,4...|(13525,[0,1,2,3,4...|(13526,[1,2,3,4,5...|(13526,[1,2,3,4,5...|[2.47532707061892...|[0.92239395348824...| 0.0|
 0|I HAVE A DATE ON ...| 1|[i, have, a, date...|(13525,[1,3,14,16...|(13525,[1,3,14,16...|(13526,[0,2,4,15,...|(13526,[0,2,4,15,...|[1.90544195305308...|[0.87050620846253...| 0.0|
 1|XXXMobileMovieClu...| 1|[xxxmobilemoviecl...|(13525,[0,4,8,11,...|(13525,[0,4,8,11,...|(13526,[0,1,5,9,1...|(13526,[0,1,5,9,1...|[0.50769578645279...|[0.62426615787796...| 0.0|
 0|Oh k...i'm watchi...| 0|[oh, k...i'm, wat...|(13525,[159,314,4...|(13525,[159,314,4...|(13526,[160,315,4...|(13526,[160,315,4...|[2.79089435542085...|[0.94218178424803...| 0.0|
 0|Eh u remember how...| 0|[eh, u, remember,...|(13525,[1,5,20,46...|(13525,[1,5,20,46...|(13526,[2,6,21,47...|(13526,[2,6,21,47...|[2.85847497069922...|[0.94575511505416...| 0.0|
 0|Fine if that's th...| 0|[fine, if, that's...|(13525,[4,5,30,59...|(13525,[4,5,30,59...|(13526,[5,6,31,60...|(13526,[5,6,31,60...|[2.79089435542085...|[0.94218178424803...| 0.0|
 1|England v Macedon...| 1|[england, v, mace...|(13525,[0,4,28,81...|(

In [49]:
has_uppercase_coeff= scaled_pipeline.stages[-1].coefficients
has_uppercase_coeff
# 1. Since the coefficient value of 'has_uppercase' is 0.92 which is a positive value, it is a feature that is positively related to an SMS being spam.

Out[ 130 ]: SparseVector(13526, {0: 0.9289, 1: 0.9174, 2: -0.7376, 4: 0.154, 10: -0.0625, 11: 0.2256, 12: 0.875, 14: -0.047, 16: 2.0119, 21: 0.3854, 25: 0.6186, 29: 0.3159, 41: 0.5569, 49: 0.1865, 50: 0.1992, 56: 1.2697, 64: 0.0684, 69: 1.4531, 82: 1.5802, 86: 0.4017, 88: 0.5251, 96: 1.0955, 97: 1.4391, 102: 1.0041, 103: 0.2397, 109: 1.3385, 110: 1.3782, 111: 0.0415, 131: 0.65, 154: 0.7224, 168: 1.3173, 170: 0.2842, 171: 0.4273, 179: 0.8028, 180: 0.6217, 181: 0.1321, 204: 0.6288, 213: 0.5852, 224: 0.6594, 234: 0.8617, 235: 0.2618, 262: 0.0258, 263: 0.1595, 274: 0.2208, 276: 0.6916, 286: 0.597, 289: 0.6877, 291: 0.4626, 302: 0.2451, 308: 0.0278, 318: 0.1918, 331: 0.2985, 334: 0.0142, 345: 0.0801, 353: 0.0593, 360: 0.3731, 383: 0.1629, 384: 0.0089, 399: 0.4152, 403: 0.2221, 428: 0.3498, 431: 0.0381, 443: 0.2669, 449: 0.7368, 454: 0.1396, 456: 0.5005, 458: 0.0099, 465: 0.5622, 483: 0.1408, 485: 0.1848, 486: 0.1238, 492: 0.1117, 497: 0.3971, 507: 0.1277, 527: 0.3345, 535: 0.1307, 542: 0.6873, 547: 0.0276, 549: 0.1408, 553: 0.4842, 578: 0.2195, 606: 0.2999, 608: 0.1159, 623: 0.115, 627: 0.0476, 628: 0.2006, 641: 0.0433, 645: 0.4791, 647: 0.2473, 658: 0.2473, 666: 0.0156, 670: 0.1412, 675: 0.1412, 685: 0.0634, 698: 0.0994, 703: 0.2858, 718: 0.6274, 721: 0.0087, 725: 0.2384, 728: 0.3661, 759: 0.0108, 766: 0.0412, 784: 0.2112, 788: 0.1715, 791: 0.0938, 793: 0.2477, 831: 0.2063, 834: 0.1537, 897: 0.03, 898: 0.0159, 907: 0.0209, 918: 0.0307, 951: 0.0075, 1080: 0.2071, 1161: 0.4536, 1168: 0.4536, 1170: 0.0093, 1182: 0.1814, 1196: 0.4536, 1197: 0.0981, 1210: 0.1772, 1278: 0.4536, 1284: 0.0387, 1352: 0.3654, 1396: 0.0927, 1462: 0.0899, 1483: 0.0491, 1550: 0.0945, 1619: 0.3918, 1624: 0.0945, 1756: 0.0325, 1816: 0.1579, 2134: 0.0747, 2300: 0.0423})

In [50]:
#Biggest positive tfidf coefficient
max_coeff = max(scaled_pipeline.stages[-1].coefficients)
ratio=(has_uppercase_coeff/max_coeff)
ratio
# 2. The ratio of the coefficient of has_uppercase to the biggest positive tfidf coefficient is 0.46.

Out[ 124 ]: 0.46170057983376228